# Import libraries

In [3]:
import requests
import bs4
import os
import time
import re

# define necessary functions

## Does Website exists?

In [10]:
def existing_url(res):

    if res.status_code!=200: #if not 200, there is any error (server, host,...)
        return 'No lyrics'
    else:
        #if response is 200,do the soup
        soup=bs4.BeautifulSoup(res.text, 'lxml')
        soup_text=soup.select_one('div>hgroup>h2').text
        #already examined error text
        error_text="We couldn't find any artists matching your query."
        #print(soup_text)
        if soup_text.lower()==error_text.lower():
            return 'Page not exists'       
    
    return 'exists'

In [5]:
#url='https://www.lyrics.com/lyric/36082255/Roxette/The+Look'
url='https://www.lyrics.com/artist/hudasldbsajdasldas'
#exist=existing_url(url)
#exist
res=requests.get(url)
#soup=bs4.BeautifulSoup(res.text, 'lxml')
#soup_text=soup.select_one('div>hgroup>h2')
#soup_text.text
exist=existing_url(res)
exist

We couldn't find any artists matching your query.


'Page not exists'

## double_handling

In [6]:
def double_handling(res,title_links):
    soup=bs4.BeautifulSoup(res.text, 'lxml')
    song_select=soup.select('td.tal.qx>strong>a')#[0:20]
    print(len(song_select))
    
    for song in song_select:
        #1. Check ( and [ and split it:
        title=re.split('\s\(|\s\[|(\s|)[\?!]',song.text)[0].lower()
        #print(title)
        #2. avoid doubles by 
        if any(title in inner_list for inner_list in title_links)==False:
            link=song['href'].replace('/','',1)
            #rint(link)
            add_pair=[title,link]
            #print(add_pair)
            title_links.append(add_pair)
            #print(title_links)
            
    
    return title_links

In [7]:
url='https://www.lyrics.com/artist/Roxette/5312'
#exist=existing_url(url)
#exist
res=requests.get(url)
liste=[]
liste=double_handling(res,liste)
print(len(liste))



1510
185


## download lyrics, save in folder

In [18]:
def download_save(newpath,url,title_links,start,end):
    print(newpath)
    mislead=0
    download=0
    for song in title_links[start:end]:
        time.sleep(1)
        lyric_url=url+song[1] #fragment of url
        response = requests.get(lyric_url)
        #do the soup
        soup = bs4.BeautifulSoup(response.text, 'lxml')
        #print(type(soup))
        try: #some songs are not linked with lyrics
            lyric_text=soup.find('pre', {'id': 'lyric-body-text'}).text
        except AttributeError: #if the link actually doesn't contain lyrics
            print(f'Link of song {song[0]} does not lead to any lyrics!')
            mislead+=1
        else: 
            #print(lyric_text)
            print(song[0])
            title = re.sub('[\W]','_',song[0]) #song[0] is title of the song
            print(title)
            #print(title,song[0])

            if not os.path.exists(newpath):  #creates folder if necessary
                os.makedirs(newpath)
            with open(newpath +'/'+title + '.txt', 'w') as response_file:  #saves the lyric-files
                response_file.write(lyric_text)
            print(title_links.index(song))
            download+=1
    
    print(f'The number of misleading links: {mislead}')
    print(f'The number of downloaded files: {download}')
    print(f'The number of all tried songs: {end-start}')

In [19]:
title = re.sub('[\W]','_','the voice') #song[0] is title of the song
title

'the_voice'

# Run the Scraping Program

In [25]:
url='https://www.lyrics.com/'
path=os.getcwd()
artist_sum=0
while artist_sum<2:
    title_links=[]
    
    #0. Input-Assignemt for which Interpret
    artist=input('The lyrics of which artist you would like to scrape?')
    artist=artist.lower()
    artist_url=url+'artist/'+artist
    print(artist_url)
    
    #request the url
    res=requests.get(artist_url)
    print(res.status_code)
    #1.existing url?
    exist=existing_url(res)
    print(exist)
    #Repeat-Condition
    if (exist=='No lyrics') or exist=='Page not exists':
        print(f'{artist} is not available. Please enter another artist.')
        continue    
    
    #2.double handling: avoid '()' and '[]' and ignore doubles
    title_links=double_handling(res,title_links) #creates a nested list (list of list [[a1,a2],[b1,b2],...])
    
    #3. check if more than 100 different songs
    if len(title_links)<100:
        print(f'{artist} has just {len(title_links)} different songs instead 100. Please enter another artist.')
        continue
    else:
        print(f'{artist} has {len(title_links)} different songs. Congrats!')
    
    #4. download lyrics and save to an individual folder
    start=int(input('Insert the # of the FIRST song you want to download'))
    end=int(input('Insert the # of the LAST song you want to download'))
    print(f'The lyrics of the {end-start} songs of {artist} are going to be saved now.')
    newpath = path+'/'+artist+'_test2_data' #for new folder
    download_save(newpath,url,title_links,start,end)
    
    artist_sum+=1

The lyrics of which artist you would like to scrape?Amy Winehouse
https://www.lyrics.com/artist/amy winehouse
200
exists
1054
amy winehouse has just 89 different songs instead 100. Please enter another artist.
The lyrics of which artist you would like to scrape?Roxette
https://www.lyrics.com/artist/roxette
200
exists
1510
roxette has 185 different songs. Congrats!
Insert the # of the FIRST song you want to download10
Insert the # of the LAST song you want to download15
The lyrics of the 5 songs of roxette are going to be saved now.
/home/christoph/OneDrive/Fortbildung_Weiterbildung_Arbeit/2022_Spiced_Data_Science/Data_Science_Course/Working_Area/04_week/00_Project/roxette_test2_data
almost unreal
almost_unreal
10
Link of song the big l does not lead to any lyrics!
Link of song the big l. does not lead to any lyrics!
Link of song crash does not lead to any lyrics!
june afternoon
june_afternoon
14
The number of misleading links: 3
The number of downloaded files: 2
The number of all tried

In [89]:
output = set()
for x in title_links:
    output.add(x[0])
print(output)
len(output)

{'everyboyd', 'my beautiful woman', 'as long as you love me', "we've got it goin' on", "it's christmas time again", 'shape of my heart', "it's gotta be you", 'you can let go', 'nobody else', "darlin'", 'new love', 'all i have to give, pt. 2', 'ok', 'got to get it', 'rebel', "that's the way i like it", "she's a dream", 'time', "quit playin' games", 'love somebody', 'i need you tonight / heaven in your eyes', 'aurora', 'make believe', 'pda', 'set adrift on memory bliss', "what's going on", 'shape of the heart', 'the one', 'how did i fall in love with you', "let's make a toast to our love", 'matches', 'on without you', 'try', 'quit playing games', 'where can we go from here', "you're the reason", 'the way it is', "i'll never brak your heart", 'lay down beside me', 'let it be me', 'song for the unloved', 'permanent stain', 'no goodbyes', 'sexbomb', 'crawling back to you', 'one phone call', "movin' on", 'treat me right', 'shoe me the meaning of being lonely', 'hey, mr, dj', "don't try this 

214